In [1]:
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from random import *
import numpy as np

In [2]:
nist = fetch_mldata('MNIST original')
mnist = nist.data
mnistnew = mnist;
mnistnew = mnistnew.reshape((70000,28,28))
mnistlabel = nist.target

In [3]:
def isBlack(x):
    if(x != 0):
        return 1
    else:
        return 0

In [18]:
def RandomGenerateRectPoints():
    point = []
    while(len(point) < 100):
        x = randint(0, 27)
        y = randint(0, 27)
        a = randint(5, 27)
        b = randint(5, 27)
        localpoint = []
        if(x<a and y<b):
            num = (a-x) * (b-y)
            if(num >= 130 and num <= 170):
                localpoint.append((x,y))
                localpoint.append((a,b))
                if(localpoint not in point):
                    point.append(localpoint)
    return point

In [19]:
points = RandomGenerateRectPoints()

In [20]:
BlackPoints = np.zeros((70000,28,28))

In [21]:
imagecount = -1
for image in mnistnew:
    imagecount+=1
    for i in range(len(image)):
        for j in range(len(image[i])):
            
            if(i==0 and j ==0):
                BlackPoints[imagecount][i][j] = isBlack(image[i][j])
            elif(i==0 and j>0):
                BlackPoints[imagecount][i][j] = BlackPoints[imagecount][0][j-1] + isBlack(image[i][j])
            elif(j==0 and i>0):
                BlackPoints[imagecount][i][j] = BlackPoints[imagecount][i-1][0] + isBlack(image[i][j])
            else:
                BlackPoints[imagecount][i][j] = BlackPoints[imagecount][i-1][j] + BlackPoints[imagecount][i][j-1] + isBlack(image[i][j]) - BlackPoints[imagecount][i-1][j-1]

In [28]:
def countofblackpt(imagecount,x,y,a,b):
    return BlackPoints[imagecount][a][b] - BlackPoints[imagecount][x][b] - BlackPoints[imagecount][a][y] + BlackPoints[imagecount][x][y]

In [29]:
def features(imagecount,x,y,a,b):
    
    p = int((a+x)/2)
    num1 = countofblackpt(imagecount,x,y,p,b) - countofblackpt(imagecount,p,y,a,b)
    
    q = int((y+b)/2)
    num2 = countofblackpt(imagecount,x,y,a,q) - countofblackpt(imagecount,x,q,a,b)
    return [num1,num2]

In [30]:
ResultMatrix = []
for i in range(70000):
    localfeatures = []
    for p in points:
        ans = features(i,p[0][0],p[0][1],p[1][0],p[1][1])
        localfeatures.append(ans[0])
        localfeatures.append(ans[1])
    ResultMatrix.append(localfeatures)
ResultMatrix = np.array(ResultMatrix)

In [32]:
print(ResultMatrix.shape)
X_train, X_test, label_train, label_test = train_test_split(ResultMatrix,mnistlabel , test_size=0.10, random_state=42)

(70000, 200)


In [34]:
from sklearn.tree import DecisionTreeClassifier
regr_2 = DecisionTreeClassifier(max_depth=15)
regr_2MN = regr_2.fit(X_train,label_train)
regr_2MN.score(X_test,label_test)

0.8838571428571429

In [38]:
from sklearn.ensemble import GradientBoostingClassifier
grd = GradientBoostingClassifier()
grd.fit(X_train,label_train)
grd.score(X_test,label_test)

0.9422857142857143